In [1]:
import pylab as pl
from sklearn import datasets, svm, metrics
#Using the premade data sets iris and digits.
#Iris contains data for plants I think and
#digits contains data from handwritten numbers.
iris = datasets.load_iris()
digits = datasets.load_digits()
#We're using the 'support vector classification' estimator, clf
#stands for classify or some such since we're going to try to classify
#what digit was written
clf = svm.SVC(gamma=0.001,C=100.)

clf.fit(digits.data[:-3],digits.target[:-3])
clf.predict(digits.data[-3])
#print(digits.target[-3])
#print(digits.images[-3])
pl.imshow(digits.images[-1], cmap=pl.cm.gray_r)

In [7]:
import pylab as pl
import numpy as np
from sklearn import datasets, svm, metrics
from datetime import datetime

iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

#Random numbers are pseudorandom, the obect in the parentheses is a seed to alter where we look at the 
#psuedo random numbers. By using the datetime object our seed changes everytime we run it(well, almost every time
#there are 10k possible numbers), so the numbers are effectively randomized.
np.random.seed(int(str(datetime.now())[-4:]))
indices = np.random.permutation(len(iris_X))


In [25]:
#Exercise 1: Supervised learning, test prediction performance on digits.
#The following tests the knn method
from sklearn import datasets, neighbors, linear_model
from sklearn.neighbors import KNeighborsClassifier

digits = datasets.load_digits()
x_digits = digits.data
y_digits = digits.target

# Use 90% as training, 10% as test data
digits_x_train = x_digits[:len(x_digits)*9/10]
digits_y_train = y_digits[:len(y_digits)*9/10]
digits_x_test = x_digits[len(x_digits)*9/10:]
digits_y_test = y_digits[len(y_digits)*9/10:]

knn = KNeighborsClassifier()
knn.fit(digits_x_train,digits_y_train)
knn.predict(digits_x_test)
knn.fit(digits_x_train,digits_y_train).score(digits_x_test,digits_y_test)

0.96111111111111114

In [26]:
from sklearn import datasets, linear_model

logistic = linear_model.LogisticRegression()

# Import the digits dataset, assign data to x, target (answers) to y
digits = datasets.load_digits()
x_digits = digits.data
y_digits = digits.target

# Use 90% as training, 10% as test data
digits_x_train = x_digits[:len(x_digits)*9/10]
digits_y_train = y_digits[:len(y_digits)*9/10]
digits_x_test = x_digits[len(x_digits)*9/10:]
digits_y_test = y_digits[len(y_digits)*9/10:]

logistic.fit(digits_x_train,digits_y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [28]:
logistic.fit(digits_x_train,digits_y_train).score(digits_x_test,digits_y_test)

0.93888888888888888

## 2nd Exercise: SVM 

In [58]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from datetime import datetime

iris = datasets.load_iris()
n=9

# Making a psuedo-random number based on current time
seed = int(str(datetime.now())[-4:])
np.random.seed(seed)

# Make a random set of indices from the length of the data
indices = np.random.permutation(len(iris.data))
xall = iris.data[indices[:len(iris.target)],:2]
yall = iris.target[indices[:len(iris.target)]]
xall = xall[yall != 0]
yall = yall[yall != 0]
# Note to self: The above doesn't work with lists in general, iris.data is a special
# numpy array. It has functionality like "ignore the indices where y=0" (e.g. y!=0)
n_sample = len(xall)

x = xall[:n_sample*9/10]  # we only take the first two features. We could
y = yall[:n_sample*9/10]   # avoid this ugly slicing by using a two-dim dataset

xtest = xall[n_sample*9/10:]
ytest = yall[n_sample*9/10:]

C = 1.0  # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C)
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
lin_svc = svm.LinearSVC(C=C)

lin_svc.fit(x,y)
lin_svc.predict(xtest)

Automatically created module for IPython interactive environment


array([2, 2, 2, 2, 2, 2, 2, 1, 2, 1])

In [66]:
print("Linear score is %s") % lin_svc.fit(x,y).score(xtest,ytest)
print("RBF score is %s") % rbf_svc.fit(x,y).score(xtest,ytest)
print("Poly score is %s") % poly_svc.fit(x,y).score(xtest,ytest)

Linear score is 0.7
RBF score is 0.6
Poly score is 0.5


In [52]:
for fig_num, kernel in enumerate(('linear', 'rbf', 'poly')):
    clf = svm.SVC(kernel=kernel, gamma=10)
    clf.fit(x, y)

    plt.figure(fig_num)
    plt.clf()
    plt.scatter(xall[:, 0], xall[:, 1], c=yall, zorder=10, cmap=plt.cm.Paired)
    
    plt.scatter(xtest[:, 0], xtest[:, 1], s=80, facecolors='none', zorder=10)
    
    plt.axis('tight')
    x_min = xall[:, 0].min()
    x_max = xall[:, 0].max()
    y_min = xall[:, 1].min()
    y_max = xall[:, 1].max()

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    Z = clf.decision_function(np.c_[XX.ravel(), YY.ravel()])
    Z = Z.reshape(XX.shape)
    plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired)
    plt.contour(XX, YY, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'],
                levels=[-.5, 0, .5])

    plt.title(kernel)
plt.show()

## Model Selection - Cross Validation

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, datasets, svm

digits = datasets.load_digits()
X = digits.data
y = digits.target

# Support Vector Machine *dot* Support Vector Classifier 'Linear'
# Other kernels: Polynomial, rbf
svc =svm.SVC(kernel='linear')

# np.logspace makes a set of evenly spaced (logarithmically) numbers
# np.logspace(min,max,n)
# min -> 10^min ; max -> 10^max ; n = number of numbers
C_s = np.logspace(-10,0,10)

# Will populate score list with the machine's score for given C
scores = list()
# Will populate score_std with std dev for error bars
scores_std = list()

for C in C_s:
    svc.C = C
    # Cross validation splits the data into 3 sets by default, then lists the scores
    # in an array or list
    this_score= cross_validation.cross_val_score(svc,X,y,n_jobs=-1)
    scores.append(np.mean(this_score))
    scores_std.append(np.std(this_score))
    
plt.figure(1, figsize=(4,3))
plt.clf()
plt.semilogx(C_s,scores,'b--')
plt.semilogx(C_s, np.array(scores)+np.array(scores_std),'b--')
plt.semilogx(C_s, np.array(scores)-np.array(scores_std),'b--')

locs,lab = plt.yticks()
plt.yticks(locs,list(map(lambda x: "%g" % x, locs)))

plt.ylabel('CV score')
plt.xlabel('Parameter C')
plt.ylim(0,1.1)
plt.show()

## Grid-search and c-v estimators

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, datasets, svm, linear_model
from sklearn.grid_search import GridSearchCV

diabetes = datasets.load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]

lasso = linear_model.Lasso()
alphas = np.logspace(-4,-.5,30)
clf = GridSearchCV(estimator=lasso,param_grid=dict(alpha=alphas),n_jobs=-1)
clf.fit(X,y)
clf.best_score_
print(clf.best_estimator_.alpha)

#clf.score(X,y)

0.0788046281567


In [42]:
i1=90
list1 = [i for i in range(203)]
for i in range(len(list1)):
    if 100.*i/len(list1)%10 < 0.02:
        print("%s/%s %s%s complete") % (i,len(list1),100*i/len(list1),"%")

0/203 0% complete
